In [1]:
import torch
import torch.nn.functional as F
import random

In [2]:
#implement classes, this is the same as what equivalent pytorch classes do
class Linear:
  
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # note: kaiming init
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
  
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    
    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0,1)
            xmean = x.mean(dim, keepdim=True) # batch mean
            xvar = x.var(dim, keepdim=True) # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

# -----------------------------------------------------------------------------------------------
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

# -----------------------------------------------------------------------------------------------
class Embedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.weight = torch.randn((num_embeddings, embedding_dim))
    
    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out
    
    def parameters(self):
        return [self.weight]
    
#-----------------------------------------------------------------------------------------------  
class Flatten:
    
    def __call__(self,x):
        self.out = x.view(emb.shape[0], -1)
        return self.out
    
    def parameters(self):
        return []

In [3]:
#setup globabl variables
words = open('names.txt','r').read().splitlines()
chars = sorted(list(set(''.join(words)))) #list of chars
stoi = {s:i+1 for i,s in enumerate(chars)}#mapping of chars to integers
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()} #map integers to chars
vocab_size = len(stoi)

In [4]:
#build the dataset out of blocks
block_size = 3

def build_dataset(words, block_size):
    '''input is a list of words and the size of each block
    output X maps block_size consecutive characters to the next character Y'''
    X, Y = [], []
    for word in words:
        context = [0]*block_size
        #print(word)
        for char in word + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] #crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

#80% of data will be used for training, 10% for hyperparam optimization, 10% for validation
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size) #training
Xdev, Ydev = build_dataset(words[n1:n2], block_size) #hyper-param dev
Xte, Yte = build_dataset(words[n2:], block_size) #testing

## set up parameters of MLP using pytorch functions

In [6]:
n_embed = 10 #dimensions in character embeddings
n_hidden = 200 #number of neurons in the hidden MLP layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator = g)

#build a 2 layer neural net to predict Ys from Xs
#first layer goes to n_hidden
# we should multiply to change standard deviation of backprop weights
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g)*0.01 #weights

#turn MLP layer to to 27 digits
W2 = torch.randn((n_hidden, vocab_size), generator=g)*0.01 #weights
b2 = torch.randn(vocab_size, generator=g)*0.01 #biases, multiple by 

#batch normalization
bngain = torch.ones((1,n_hidden))
bnbias = torch.zeros((1,n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [11]:
#training
max_steps = 10000
batch_size = 32

for i in range(max_steps):
    #construct minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] #batches
    
    #forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1) #concatenate the vectors so you can do matrix multip
    pre_activations = embcat@W1
    
    #Batch normalization: want activations in hidden layer to be normal in the beginning, but then free to change!
    #use mean and standard deviation of the whole system, not just the batch, so we keep a running tally
    bmean = pre_activations.mean(0, keepdim=True)
    bstd = pre_activations.std(0, keepdim=True)
    pre_activations = (bngain*((pre_activations - bmean)/bstd)) + bnbias
    
    first_activations = torch.tanh(pre_activations)
    logits = first_activations@W2 + b2 #output
    loss = F.cross_entropy(logits, Yb) #compute loss 
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1 #if i < 100000 else 0.01  #change learning rate as you surpass a certain level on training
    for p in parameters:
        p.data += -lr*p.grad 

In [12]:
#loss on testing data
#forward pass
training_emb = C[Xte]
training_embcat = training_emb.view(training_emb.shape[0], -1) #concatenate the vectors so you can do matrix multip
training_first_activations = torch.tanh(training_embcat@W1)
training_logits = training_first_activations@W2 + b2 #output
training_loss = F.cross_entropy(training_logits, Yte) #compute loss
training_loss.data

tensor(2.7487)

## set up parameters of MLP using our classes

In [5]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

layers = [Embedding(vocab_size, n_embd),
          Flatten(),
          Linear(n_embd*block_size, n_hidden, bias = False), 
          BatchNorm1d(n_hidden), Tanh(),
          Linear(n_hidden, vocab_size)
]

with torch.no_grad():
    # last layer: make less confident
    layers[-1].weight *= 0.1

parameters = [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

12097


In [16]:
# same optimization as last time
max_steps = 10000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
    
    # forward pass
    x = Xb
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb) # loss function
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
 
    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x960 and 30x200)

In [10]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        # forward pass the neural net
        emb = C[torch.tensor([context])] # (1,block_size,n_embd)
        x = emb.view(emb.shape[0], -1) # concatenate the vectors
        for layer in layers:
            x = layer(x)
        logits = x
        probs = F.softmax(logits, dim=1)
        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the special '.' token, break
        if ix == 0:
            break
            
        print(''.join(itos[i] for i in out)) # decode and print the genera

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0